In [1]:
# A bit of setup
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
# The code was removed by DSX for sharing.

,Material,Usura_mat_g,Durezza_Disco,Comprex_LP,Comprex_LR,Grindo_LP,Grindo_LR,PistoneCpx,Inerzia,Vmax,...,183,184,185,186,187,188,189,190,191,192
0,1.000000,0.900208,0.880503,0.673423,0.677704,0.129032,0.062284,1,0.272727,0.434783,...,0.539481,0.534210,0.740648,0.438179,0.516201,0.320636,0.623794,0.298908,0.689731,0.515836
1,0.997764,0.919958,0.805031,0.675676,0.631347,0.206452,0.159170,1,0.272727,0.434783,...,0.675487,0.691069,0.701569,0.363226,0.450055,0.331382,0.711773,0.343537,0.808774,0.616340
2,0.996452,0.980249,0.440252,0.801802,0.779249,0.218280,0.156863,1,0.272727,0.434783,...,0.539481,0.534210,0.740648,0.438179,0.516201,0.320636,0.623794,0.298908,0.689731,0.515836
3,0.996452,0.977131,0.427673,0.774775,0.854305,0.246237,0.222607,1,0.272727,0.434783,...,0.539481,0.534210,0.740648,0.438179,0.516201,0.320636,0.623794,0.298908,0.689731,0.515836
4,0.995529,0.944906,0.786164,0.641892,0.635762,0.118280,0.071511,1,0.272727,0.434783,...,0.613546,0.546417,0.741318,0.444433,0.520374,0.323024,0.615110,0.295961,0.698293,0.475325


In [ ]:
print "Variable definition and assignment:"

array_data_1 = df_data_1.values
y_data_1 = df_data_1.index

D =  array_data_1.shape[1]# number of features
K = array_data_1.shape[0] # number of classes
X = array_data_1
y = df_data_1.index
num_examples = array_data_1.shape[0]

print "Shape of input matrix X: " + str(X.shape)
print "Shape of target vector y: " + str(y.shape)
print "Number of target classes K: " + str(K)
print "Number of features D: " + str(D)
print "Number of samples num_samples: " + str(num_examples)

h = 150 # size of hidden layer, the new number of features per sample
W = 0.01 * np.random.randn(D,h)
b = np.zeros((1,h))
W2 = 0.01 * np.random.randn(h,K)
b2 = np.zeros((1,K))

print "Number of hidden units h: " + str(h)
print "Shape of the input-hidden weights matrix W: " + str(W.shape)
print "Shape of the input-hidden bias b: " + str(b.shape)
print "Shape of the hidden-output weights matrix W2: " + str(W2.shape)
print "Shape of the hidden-output bias b2: " + str(b2.shape)

Variable definition and assignment:
Shape of input matrix X: (651, 235)
Shape of target vector y: (651,)
Number of target classes K: 651
Number of features D: 235
Number of samples num_samples: 651
Number of hidden units h: 150
Shape of the input-hidden weights matrix W: (235, 150)
Shape of the input-hidden bias b: (1, 150)
Shape of the hidden-output weights matrix W2: (150, 651)
Shape of the hidden-output bias b2: (1, 651)


In [ ]:
# some hyperparameters
step_size = 1e-2
reg = 1e-3 # regularization strength
training_losses = []

for i in xrange(200000):
  
    # evaluate class scores, [N x K]
    hidden_layer = np.maximum(0, np.dot(X, W) + b) # note, ReLU activation
    scores = np.dot(hidden_layer, W2) + b2
  
    # compute the class probabilities
    exp_scores = np.exp(scores)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) # computing for each sample the probability of being a certain sample
  
    # compute the loss: average cross-entropy loss and regularization
    corect_logprobs = -np.log(probs[range(num_examples),y])
    data_loss = np.sum(corect_logprobs)/num_examples
    reg_loss = 0.5*reg*np.sum(W*W) + 0.5*reg*np.sum(W2*W2)
    loss = data_loss + reg_loss
    training_losses.append(loss)
    
    if i % 1000 == 0:
        print "iteration %d: loss %f" % (i, loss)
  
    # compute the gradient on scores
    dscores = probs
    dscores[range(num_examples),y] -= 1
    dscores /= num_examples
  
    # backpropate the gradient to the parameters
    # first backprop into parameters W2 and b2
    dW2 = np.dot(hidden_layer.T, dscores)
    db2 = np.sum(dscores, axis=0, keepdims=True)
    # next backprop into hidden layer
    dhidden = np.dot(dscores, W2.T)
    # backprop the ReLU non-linearity
    dhidden[hidden_layer <= 0] = 0
    # finally into W,b
    dW = np.dot(X.T, dhidden)
    db = np.sum(dhidden, axis=0, keepdims=True)
  
    # add regularization gradient contribution
    dW2 += reg * W2
    dW += reg * W
  
    # perform a parameter update
    W += -step_size * dW
    b += -step_size * db
    W2 += -step_size * dW2
    b2 += -step_size * db2

plt.plot(training_losses)
plt.title("Plot loss trend:", fontsize=30)
plt.xlabel("iteration", fontsize=20)
plt.ylabel("loss", fontsize=20)

iteration 0: loss 6.485269


In [ ]:
# evaluate training set accuracy
hidden_layer = np.maximum(0, np.dot(X, W) + b)
scores = np.dot(hidden_layer, W2) + b2
predicted_class = np.argmax(scores, axis=1)
print 'training accuracy: %.2f' % (np.mean(predicted_class == y))